In [ ]:
import pandas as pd, numpy as np, matplotlib.pyplot as plt
import cv2 as cv, os
from PIL import Image
from sklearn.cluster import KMeans
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Flatten, Dense, Dropout, BatchNormalization
from keras.models import Model
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns, os
import tensorflow as tf
from tensorflow.keras.layers import Flatten, Dense, Dropout, BatchNormalization, Input, MaxPooling2D, Conv2D
from keras.models import Model
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau, LearningRateScheduler

In [ ]:
dir_model = "/kaggle/input/models/tensorflow2/default/1/models"
models = os.listdir(dir_model)
folder_model = [f'{dir_model}/{model}' for model in models]
folder_model

In [ ]:
def testingModel(dir_image_test, dir_model):
    print('Hasil dari: ', dir_model, '\n\n')
    original_data = ImageDataGenerator(rescale=1./255)
    test_data = original_data.flow_from_directory(dir_image_test, target_size = (224, 224), batch_size = 32, class_mode="categorical", shuffle=False)

    # Load the model from an .h5 file
    model = tf.keras.models.load_model(dir_model)

    # Print the model summary to verify it loaded correctly
    model.summary()


    predictions = model.predict(test_data)

    # Get the predicted class labels
    predicted_labels = np.argmax(predictions, axis=1)

    # Get the true class labels
    true_labels = test_data.classes

    # Compute confusion matrix
    conf_matrix = confusion_matrix(true_labels, predicted_labels)

    # Plot confusion matrix using seaborn
    plt.figure(figsize=(8, 6))
    sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues',
                xticklabels=test_data.class_indices.keys(),
                yticklabels=test_data.class_indices.keys())
    plt.xlabel('Predicted')
    plt.ylabel('True')
    plt.title('Confusion Matrix')
    plt.show()

    # Print classification report
    class_names = list(test_data.class_indices.keys())
    print(classification_report(true_labels, predicted_labels, target_names=class_names))
    print(f'{dir_model} telah diperoleh hasilnya\n\n\n')

In [ ]:
for mdl in folder_model:
    testingModel('/kaggle/input/test-data-preprocess', mdl)

In [ ]:
def load_image(image_path):
    img = cv.imread(image_path)
    img = cv.cvtColor(img, cv.COLOR_BGR2RGB) 
    
    img_resized = cv.resize(img, (224, 224))  # Resize gambar ke 224x224

    return img_resized

def segment_images_kmean(image, n_clusters):
    pixels = image.reshape(-1, 3)  

    kmeans = KMeans(n_clusters=n_clusters, random_state=42)
    kmeans.fit(pixels)

    labels = kmeans.labels_
    centers = kmeans.cluster_centers_.astype(int) 

    segmented_image = centers[labels].reshape(image.shape)  
    return segmented_image

def image_preprocessing(image_path):
    img = load_image(image_path)
    img = segment_images_kmean(img, 5)
    img = np.expand_dims(img, axis=0)
    return img

def display(img_with_batch):
    img_without_batch = np.squeeze(img_with_batch, axis=0)

    # Tampilkan gambar menggunakan matplotlib
    plt.imshow(img_without_batch)
    plt.axis('off')  # Menghilangkan sumbu
    plt.show()
    
def pred(mdl, img):
    predictions = mdl.predict(img)

    # Get the predicted class labels
    labels = np.argmax(predictions, axis=1)
    return labels

In [ ]:
path_test = '/kaggle/input/my-other-test/other-test'
for im in os.listdir(path_test):
    img = image_preprocessing(os.path.join(path_test, im))
    display(img)
    labels = pred(model, img)
    predicted_labels = 'cadar' if labels == 0 else 'masking' if labels == 1 else 'unmasking'
    print(f'Hasil prediksi -> {labels}: {predicted_labels}')
    

In [ ]:
img = image_preprocessing('/kaggle/input/unmask/not-mask.jpg')
pred(model, img)

In [ ]:
display(img)